In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/TFG/Pruebas ejecucion

/content/drive/MyDrive/TFG/Pruebas ejecucion


In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 21 not upgraded.
Need to get 430 MB of archives.
After this operation, 1,153 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 5s (81.8 MB/s)
(Reading database ... 128276 files and directories currently installed.)
Removing libcudnn8-dev (8.7.0.84-1+cuda11.8) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.7.0.84-1+cuda11.8 to 8.1.0.77-1+cuda11.2
(Reading database ... 128243 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.7

In [ ]:
from data_preprocessing import create_datasets, text_vectorization, process_text
from model import Transformer,  CustomSchedule, masked_loss, masked_acc

import tensorflow as tf

from tensorflow import keras
import numpy as np
import pandas as pd

In [ ]:
def create_metrics(history):
    try:
        metrics = pd.read_csv('metrics.csv')
    except:
        metrics = pd.DataFrame(columns = ['loss', 'masked_acc', 'val_loss', 'val_masked_acc'])

    new_metrics = pd.DataFrame(history.history)
    metrics = pd.concat([metrics, new_metrics], ignore_index = True)

    metrics.to_csv('metrics.csv', index = False)

In [ ]:
# utils.init()
# context_text_processor = utils.context_text_processor
# target_text_processor = utils.target_text_processor

In [ ]:
train, val, test = create_datasets()
context_text_processor, target_text_processor = text_vectorization(train)

# print(context_text_processor.get_vocabulary()[:10])
# print(target_text_processor.get_vocabulary()[:10])

train_ds = train.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)
val_ds = val.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)
test_ds = val.map(lambda x, y: process_text(x, y, context_text_processor, target_text_processor), tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
#! Model
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [ ]:
transformer = Transformer(
    num_layers = num_layers,
    d_model = d_model,
    num_heads = num_heads,
    dff = dff,
    input_vocab_size = context_text_processor.vocabulary_size(),
    target_vocab_size = target_text_processor.vocabulary_size(),
    dropout_rate = dropout_rate,
)

In [ ]:
#! Loss and Optimizer
learning_rate = CustomSchedule(d_model)
my_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
#! Compile
transformer.compile(
    loss = masked_loss,
    optimizer=my_optimizer,
    metrics = [masked_acc],
)

In [ ]:
history = transformer.fit(
    train_ds,
    epochs = 10,
    validation_data = val_ds,
)

Epoch 1/10
820/820 [==============================] - 337s 317ms/step - loss: 5.7152 - masked_acc: 0.2069 - val_loss: 4.3897 - val_masked_acc: 0.2840
Epoch 2/10
820/820 [==============================] - 199s 242ms/step - loss: 4.0151 - masked_acc: 0.3142 - val_loss: 3.5984 - val_masked_acc: 0.3494
Epoch 3/10
820/820 [==============================] - 185s 225ms/step - loss: 3.3867 - masked_acc: 0.3675 - val_loss: 3.0557 - val_masked_acc: 0.4087
Epoch 4/10
820/820 [==============================] - 180s 220ms/step - loss: 2.9118 - masked_acc: 0.4244 - val_loss: 2.7124 - val_masked_acc: 0.4575
Epoch 5/10
820/820 [==============================] - 183s 223ms/step - loss: 2.6236 - masked_acc: 0.4638 - val_loss: 2.5150 - val_masked_acc: 0.4888
Epoch 6/10
820/820 [==============================] - 184s 225ms/step - loss: 2.4144 - masked_acc: 0.4941 - val_loss: 2.3398 - val_masked_acc: 0.5202
Epoch 7/10
820/820 [==============================] - 180s 219ms/step - loss: 2.2335 - masked_acc: 0

In [ ]:
create_metrics(history)

In [ ]:
# import io
# import json

In [ ]:
# context_tokenizer_json = context_text_processor.
# with io.open('context_tokenizer.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(context_tokenizer_json, ensure_ascii=False))

In [ ]:
# Save model
transformer.save('my_model_21032023')

# Save text processors
# with open('context_processor.pickle', 'wb') as handle:
#   pickle.dump(context_text_processor, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('target_text_processor.pickle', 'wb') as handle:
#   pickle.dump(target_text_processor, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: ignored

In [ ]:
reconstructed_model = keras.models.load_model("my_model_16032023", custom_objects={ 'CustomSchedule' : CustomSchedule, 'masked_loss': masked_loss, 'masked_acc': masked_acc })

In [ ]:
# np.testing.assert_allclose(
#     transformer.predict(val_ds), reconstructed_model.predict(val_ds)
# )

In [ ]:
# reconstructed_model.fit(
#     test_ds,
#     epochs = 3,
#     validation_data = val_ds,
# )

In [ ]:
def print_translation(sentence, translated_sentence, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {translated_sentence}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
def detokenize_sentence(processor, sentence):
  output_sentence = tf.constant('')
  vocab = processor.get_vocabulary()

  for token in sentence:
    output_sentence = tf.strings.join([output_sentence, vocab[token]], separator=' ')
                                        
  sentence = output_sentence.numpy().decode('utf-8')
  sentence = sentence.replace('[START]', '').replace('[END]', '').strip()
  
  return sentence

In [ ]:
def translate(sentence, context_processor, target_processor, model, max_tokens=128):
  # Convert the sentence to tensor
  sentence = tf.constant(sentence)
  assert isinstance(sentence, tf.Tensor)

  # Give the correct shape
  if len(sentence.shape) == 0:
    sentence = sentence[tf.newaxis]
  
  # Tokenize the sentence
  sentence = context_processor(sentence).to_tensor()

  # Input for the encoder
  encoder_input = sentence
  
  # Input for the decoder
  start_end_tokens = target_processor([''])[0]
  start_token = start_end_tokens[0][tf.newaxis]
  end_token = start_end_tokens[-1][tf.newaxis]

  # Convert decoder sentence to TensorArray for feed the model
  output_tensor = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
  output_tensor = output_tensor.write(0, start_token)
  
  # seq_to_seq generation
  for i in tf.range(max_tokens):
    # Prepare the output tensor
    output = tf.transpose(output_tensor.stack())
    
    # Get the model predictions
    predictions = model([encoder_input, output], training=False)
    
    # Select the last token from the seq_len dimension
    predictions = predictions[:, -1, :] # Shape (batch_size, 1, vocab_size)
    # Get the most probable next token
    predicted_id = tf.argmax(predictions, axis=-1)

    # Concatenate the predicted token to the output sentence
    output_tensor = output_tensor.write(i+1, predicted_id) #TODO: en el tuto es predicted_id[0]

    # Check if the prediction is fully generated with the last token
    if predicted_id == end_token:
      break
  
  # Convert the output to the correct shape
  output = tf.transpose(output_tensor.stack()) # Shape (1, tokens_generated)

  # Detokenize the output sentence
  text = detokenize_sentence(target_processor, output[0])
  return text

In [ ]:
sentence = 'I want to be at home today'
ground_truth = 'Quiero estar en casa'

translated_text = translate(sentence, context_text_processor, target_text_processor, transformer)
print_translation(sentence, translated_text, ground_truth)

Input:         : I want to be at home today
Prediction     : quiero estar en casa
Ground truth   : Quiero estar en casa


In [ ]:
sentence = 'I like to drink water in the park'
ground_truth = 'Me gusta beber agua en el parque'

translated_text = translate(sentence, context_text_processor, target_text_processor, transformer)
print_translation(sentence, translated_text, ground_truth)

Input:         : I like to drink water in the park
Prediction     : me gusta beber agua en el parque
Ground truth   : Me gusta beber agua en el parque


In [ ]:
sentence = 'university has taught me that life is not only about technical knowledge, but also about meeting people who can help you in the future.'
ground_truth = 'la universidad me ha enseñado que en la vida no solo importan los conocimientos técnicos, sino también conocer personas que te puedan ayudar en el futuro'

translated_text = translate(sentence, context_text_processor, target_text_processor, transformer)
print_translation(sentence, translated_text, ground_truth)

Input:         : university has taught me that life is not only about technical knowledge, but also about meeting people who can help you in the future.
Prediction     : la universidad ha [UNK] que la vida no es solo acerca de conocimiento tecnica , sino tambien sobre la reunion personas que pueden ayudar en el futuro .
Ground truth   : la universidad me ha enseñado que en la vida no solo importan los conocimientos técnicos, sino también conocer personas que te puedan ayudar en el futuro
